In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pydicom
import numpy as np
import os
import glob
from tqdm import tqdm
import warnings
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.initializers import GlorotUniform, HeNormal
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.losses import BinaryCrossentropy

import SimpleITK as sitk

%matplotlib inline
np.random.seed(42)
tf.random.set_seed(42)

2024-08-13 05:48:48.906001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 05:48:48.906178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 05:48:49.064051: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Use this cell to empty the output folder
# !rm -rf /kaggle/working/*

In [3]:
root_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
train_csv_path = os.path.join(root_path, 'train.csv')
train_csv = pd.read_csv(train_csv_path)
coordinates_path = os.path.join(root_path, 'train_label_coordinates.csv')
df_coor = pd.read_csv(coordinates_path)

In [4]:
# read MRI volume
# INPUT: A folder consisting of MRI slices
# for example: /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/100206310/1012284084"
# OUTPUT: Image 
def readMRIVolume(mri_volume_path):
    reader = sitk.ImageSeriesReader()
    dicom_files = reader.GetGDCMSeriesFileNames(mri_volume_path)
    reader.SetFileNames(dicom_files)
    retrieved_mri_volume = reader.Execute()
    return retrieved_mri_volume

In [5]:
# Resample images to uniform voxel spacing
# INPUT: Image
# OUTPUT: Image
def resample_image(input_volume, out_spacing=[1, 1, 1]):
  
    original_spacing = input_volume.GetSpacing()
    original_size = input_volume.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(input_volume.GetDirection())
    resample.SetOutputOrigin(input_volume.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(input_volume.GetPixelIDValue())

    return resample.Execute(input_volume)

In [6]:
# Resize images to fixed spatial resolution in pixels
# INPUT: Image
# OUTPUT: Image
def resize_image(input_volume, output_size=320):
    num_axial_slices = int(input_volume.GetSize()[-1])
    output_size = [output_size, output_size, num_axial_slices]
    scale = np.divide(input_volume.GetSize(), output_size)
    spacing = np.multiply(input_volume.GetSpacing(), scale)
    transform = sitk.AffineTransform(3)
    resized_volume = sitk.Resample(input_volume, output_size, transform, sitk.sitkLinear, input_volume.GetOrigin(),
                                  spacing, 
    input_volume.GetDirection())
    return resized_volume

In [7]:
# INPUT: Image
# OUTPUT: Numpy array
def extract_slices(image_volume):
    image_array = sitk.GetArrayFromImage(image_volume)
    return image_array

In [8]:
def load_dicom_stack(dicom_folder_path):
    try:
        mri_image_vol = readMRIVolume(dicom_folder_path) # this is excpected to throw exception
        resampled_img = resample_image(mri_image_vol)
        resized_img = resize_image(resampled_img) 
        img_numpy_arr =  extract_slices(resized_img) # someNumber * 320 * 320
        img_middle_slice = img_numpy_arr[len(img_numpy_arr) // 2] # 320 * 320
        return img_middle_slice
    except Exception as e:
        print(f"An exception occured occurred: {e}")
        return np.zeros((320, 320))

In [9]:
# INPUTS: 
#    meta_df (Pandas dataframe): metadata of Images
#.   image_path (String): The path to Images
#    OUTPUT: A Numpy array of size (data_points * 3 * 320 * 320)

# TRAINING: 
#   meta_df = train_meta_df
#   image_path = train_images_path

# NOTE: not using tqdm because print() and tqdm.write() is causing problems
def create_dataset(meta_df, image_path, data_points_input=None):
    count_df = meta_df.groupby('study_id').size().reset_index(name='count')
    
    if data_points_input is not None:
        count_df = count_df.head(data_points_input)
        
    data_points = count_df.shape[0]
    data_set = np.empty((data_points, 3, 320, 320)) 
    
    for i in range(data_points):
        print(f"{i+1}th iteration out of {data_points}")
        study_ID = count_df.study_id.iloc[i]
        study = meta_df.loc[meta_df.study_id == study_ID]

        for row in study.itertuples():
            path = os.path.join(image_path, str(row.study_id), str(row.series_id))

            if row.series_description == "Sagittal T2/STIR":
                sag_t2_mid_slice = load_dicom_stack(path) # 320 * 320
            elif row.series_description == "Sagittal T1":
                sag_t1_mid_slice = load_dicom_stack(path) # 320 * 320
            elif row.series_description == "Axial T2":
                ax_t2_mid_slice = load_dicom_stack(path) # 320 * 320

        curr_set = np.stack((sag_t2_mid_slice, sag_t1_mid_slice, ax_t2_mid_slice), axis=0) # (3*320*320)
        data_set[i] = curr_set 
            
    return data_set

In [10]:
train_description_path = os.path.join(root_path, 'train_series_descriptions.csv')
train_meta_df = pd.read_csv(train_description_path)
train_meta_df = train_meta_df.drop_duplicates(subset=['study_id', 'series_description'])
train_images_path = os.path.join(root_path, 'train_images')

In [11]:
def load(path):
    shape = (1975, 320, 320, 3)
    data_set = np.memmap(path, mode='r', shape=shape)
    return data_set

[[3 3 4 ... 0 0 0]
 [3 5 7 ... 0 0 0]
 [3 5 9 ... 0 0 0]
 ...
 [1 3 4 ... 0 0 0]
 [1 3 4 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
 
 

In [12]:
def create_labels():
    train_csv_copy = train_csv.copy()

    new_columns = []
    severities = ['Normal/Mild', 'Moderate', 'Severe']

    for disease in train_csv_copy.columns[1:]:
        for severity in severities:
            new_col_name = f"{disease}_{severity}"
            new_columns.append(new_col_name)
            train_csv_copy[new_col_name] = 0
            train_csv_copy.loc[train_csv_copy[disease] == severity, new_col_name] = 1

    train_csv_copy = train_csv_copy.drop(columns=train_csv_copy.columns[0:26])
    # print(train_csv_copy[0])
    train_csv_copy = train_csv_copy.to_numpy()
    # print(train_csv_copy)
    
    return train_csv_copy


In [13]:
# INPUT: numpy datasets 
# OUTPUT: tf datasets train_set, val_set
# !!! num_samples=1975
def create_tf_dataset(data_set, labels, num_samples=1975, train_ratio=0.8, batch_size=32, seed=42):
    data_set_tf = tf.data.Dataset.from_tensor_slices(data_set)
    labels_tf = tf.data.Dataset.from_tensor_slices(labels)

    # Combine the inputs and labels
    combined_set = tf.data.Dataset.zip((data_set_tf, labels_tf))
    
    # Shuffle and split the dataset
    # Lets skip shuffling for now
    # combined_set = combined_set.shuffle(buffer_size=num_samples, seed=seed)
    train_size = int(num_samples * train_ratio)
    train_set = combined_set.take(train_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_set = combined_set.skip(train_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return train_set, val_set


In [14]:
def create_model(input_shape):
    print("Creating the model....")
    model_input = tf.keras.Input(shape=input_shape, name='input')

    base_model = EfficientNetV2B0(
        include_top = False,
        weights = '/kaggle/input/rsna-dataset-numpy-complete/efficientnetv2-b0_notop.h5', 
        input_shape = input_shape
    )
    base_model.trainable = False
    
    base_model_features = base_model(model_input, training=False)
    base_model_features = layers.GlobalAveragePooling2D()(base_model_features)
    # base_model_features = layers.Dropout(0.2)(base_model_features) 
    
    output = keras.layers.Dense(75, activation='softmax', kernel_initializer=GlorotUniform(), name='output')(base_model_features)
    
    model = Model(inputs=[model_input], outputs=[output])
    
    num_layers = len(base_model.layers)
    # print(f"Number of layers in the base model: {num_layers}")

    return model, base_model

In [15]:
# Train the model
def train_test_save_model(train_dataset, validation_data, epochs_to_train):
    
    input_shape = (320, 320, 3)
    model, base_model = create_model(input_shape)
    print(model.summary(show_trainable=True))
    
    print("Compiling the model....")
    model.compile(
        optimizer=keras.optimizers.Adam(), 
        loss=keras.losses.BinaryCrossentropy(), 
        metrics=['accuracy']
    )
    
    print("Training the model....")
    history = model.fit(
        train_dataset,
        validation_data=validation_data,
        epochs=epochs_to_train,  # Number of epochs to train
        verbose=1  # Verbosity mode
    )

    print("Evaluating the model....")
    results = model.evaluate(validation_data)
    print(f"Validation results - {results}")

    print("Saving the model....")
    model.save('non_fine_tuned_model.h5')
    
    return model, base_model;

In [16]:
# Fine tune the model
# there are 270 layers in efficientNet_v2_b0
def train_test_save_fine_tune_model(train_dataset, validation_data, epochs_to_train, epochs_to_fine_tune,layers_to_unfreeze):
    model, base_model = train_test_save_model(train_dataset, validation_data, epochs_to_train)
    
    print("Starting the fine tuning process....")
    
    for layer in base_model.layers[-layers_to_unfreeze:]:
        layer.trainable = True
    # base_model.trainable = True 
    
    model.summary(show_trainable=True)
    
    model.compile(
        optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'],
    )

    print("Fitting the end-to-end model")
    model.fit(train_dataset, epochs=epochs_to_fine_tune, validation_data=validation_data)
    
    print("Evaluating the model....")
    results = model.evaluate(validation_data)
    print(f"Validation results - {results}")
    
    print("Saving the model....")
    model.save('fine_tuned_model_one_layer_unfrozen.h5')

## Testing

In [17]:
# Path to all the files
test_images_path = os.path.join(root_path, 'test_images')
test_description_path = os.path.join(root_path, 'test_series_descriptions.csv')
test_meta_df = pd.read_csv(test_description_path)
test_meta_df = test_meta_df.drop_duplicates(subset=['study_id', 'series_description'])

In [18]:
def predict(model):
    test_data_set = create_dataset(test_meta_df, test_images_path)
    test_data_set = np.transpose(test_data_set, (0, 2, 3, 1))
    test_tf = tf.data.Dataset.from_tensor_slices(test_data_set)
    test_tf = test_tf.batch(32)
    predictions = model.predict(test_tf)
    
    return predictions

## Creation of the submission file

In [19]:
def create_submission(predictions):
    # Create the severity level columns
    normal_mild_col = []
    moderate_col = []
    severe_col = []

    for curr_75_element_array in predictions:
        arrays = [normal_mild_col, moderate_col, severe_col]
        for i, element in enumerate(curr_75_element_array):
            arrays[i % 3].append(element) 

    normal_mild_col = np.array(normal_mild_col).flatten()
    moderate_col = np.array(moderate_col).flatten()
    severe_col = np.array(severe_col).flatten()
    
    # Create the row id column
    row_id_col = []
    test_meta_df_copy = test_meta_df.copy()
    unique_study_ids = test_meta_df_copy[['study_id']].drop_duplicates().reset_index(drop=True)
    diseases_array = train_csv.copy().columns[1:].to_numpy()
    for study_id in unique_study_ids['study_id']:
        for disease in diseases_array:
            # row_id_col.append({'row_id': f'{study_id}_{disease}'})
            row_id_col.append(f'{study_id}_{disease}')

    data = {
        'row_id': row_id_col,
        'normal_mild': normal_mild_col,
        'moderate': moderate_col,
        'severe': severe_col
    }
    submission_df = pd.DataFrame(data)
    submission_df.to_csv('submission.csv', index=False)

### The following cell contains the data creation code, which runs only once 

In [20]:
def np_dataset_creation():
    data_set = create_dataset(train_meta_df, train_images_path) # 5922 * 3 * 320 * 320
    data_set = np.transpose(data_set, (0, 2, 3, 1))  # 5922 * 320 * 320 * 3
    print(f"shape: {data_set.shape}")
    np.save('numpy_4d_npy_array', data_set)
    
# np_dataset_creation()

In [21]:
def load_data_set():
    data_set_path = '/kaggle/input/rsna-dataset-numpy-complete/numpy_4d_npy_array.npy'
    data_set = load(data_set_path)
    print(data_set.shape)
    return data_set


In [22]:
def complete_training_workflow(data_set, epochs_to_train, epochs_to_fine_tune, layers_to_unfreeze):
    labels = create_labels()
    train_dataset, validation_data = create_tf_dataset(data_set, labels)
    train_test_save_model(train_dataset, validation_data, epochs_to_train)

data_set = load_data_set()
complete_training_workflow(data_set, epochs_to_train=10, epochs_to_fine_tune=5,layers_to_unfreeze=0)

(1975, 320, 320, 3)
Creating the model....


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input (InputLayer)          │ (None, 320, 320, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ efficientnetv2-b0           │ (None, 10, 10, 1280)  │  5,919,312 │   N   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d    │ (None, 1280)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ output (Dense)              │ (None, 75)            │     96,075 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 6,015,387 (22.95 MB)

 Trainable params: 96,075 (375.29 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

None
Compiling the model....
Training the model....
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.0314 - loss: 0.4620 - val_accuracy: 0.0127 - val_loss: 0.3629
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.0617 - loss: 0.3732 - val_accuracy: 0.1544 - val_loss: 0.3618
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.1566 - loss: 0.3713 - val_accuracy: 0.2937 - val_loss: 0.3619
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.1972 - loss: 0.3717 - val_accuracy: 0.7089 - val_loss: 0.3620
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.2459 - loss: 0.3709 - val_accuracy: 0.7013 - val_loss: 0.3620
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.2708 - loss: 0.3709 - val_accuracy: 0.6481 - val_loss: 0.3619
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.2967 - loss: 0.3707 - val_accuracy: 0.8000 - val_loss: 0.3618
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accura

In [23]:
# !! load function accesses from the output, not the input. Submitting doesn't have access to output
def complete_testing_workflow():
    model_path = '/kaggle/input/eff_v2b0_frozen_10_epochs/keras/default/2/eff_v2b0_frozen_layer_10_epochs.h5'
    model = tf.keras.models.load_model(model_path)
    print("predicting...")
    predictions = predict(model)
    
    print("creating submission...")
    create_submission(predictions)
    print("submission created")

complete_testing_workflow()

predicting...
1th iteration out of 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
creating submission...
submission created
